

---

## **📌 Route Optimization Project using Clustering and VSP Algorithms**  
**Author:** Pedro Miguel Figueroa Domínguez  

### **📖 Introduction**  
This project is designed to **optimize visit routes** to different geographical locations using **clustering algorithms and the Traveling Salesman Problem (TSP) solution**. The process consists of two main steps:  

1. **Clustering Points:**  
   - A clustering algorithm groups locations into clusters with a **maximum of 281 points** each.  
   - These clusters are assigned to different **agents** to distribute the workload.  

2. **Route Optimization (TSP):**  
   - The **Nearest Neighbor Algorithm** is used to determine the optimal visiting order within each cluster.  
   - The route is sorted based on the **shortest distance between points**.  

### **⚙️ Prerequisites**  
Before running the notebook, install the required Python libraries:  

```python
!pip install pandas googlemaps ortools folium openpyxl
pip install folium pandas numpy scikit-learn geopy
```


### **🌍 Getting Coordinates (If Not Available in Your Data)**  
If your dataset does not contain **latitude and longitude coordinates**, you can extract them using the **Google Maps API**.  

#### **Steps to Obtain Coordinates**  

1. **Download an Address Dataset (if needed)**  
   - If you only have business names and addresses, your file should have structured address information.  
   - Recommended format: CSV or Excel with columns like **Business Name, Address, City, Country**.  

2. **Get a Google Maps API Key**  
   - Go to [Google Cloud Console](https://console.cloud.google.com/)  
   - Enable the **Geocoding API**  
   - Generate an **API Key**  

3. **Insert the API Key into the Code**  
   - The script will use this API key to query Google Maps and retrieve latitude/longitude.  

4. **Run the Script to Convert Addresses to Coordinates**  
   - The code will process each address and store the coordinates for later use.  

### **🚀 How to Use This Project**  

1. **Load the Data**  
   - Ensure your dataset includes either **coordinates** or **addresses** (if you plan to extract them).  
   - If missing coordinates, use the Google Maps API method to obtain them.  
   - The script will automatically load and process the dataset into a **DataFrame**.  

2. **Run the Route Optimization Algorithm**  
   - The script will create clusters of **maximum 281 points** each.  
   - It will then apply the **TSP algorithm** to determine the optimal visiting order.  

3. **Generate the CSV File and Visualize the Route**  
   - Enter the **agent number** to generate a **route table**.  
   - The script will generate a CSV file with the optimized route.  
   - A **map will be displayed**, showing the numbered points in the correct order.  

### **📌 Key Features**  
✔ **Efficient clustering** to distribute locations into manageable groups.  
✔ **Route optimization** using the **Nearest Neighbor TSP algorithm**.  
✔ **Automatic CSV generation** with sorted visiting orders.  
✔ **Interactive map visualization** with numbered points.  
✔ **Automatic coordinate retrieval** if not available.  

### **🎯 Expected Results**  
- **A well-organized table** displaying the optimal visiting order.  
- **An interactive map** showing points numbered in sequence.  
- **A CSV file** with the ordered route for each agent.  

### **📊 Excel File Structure**  
The Excel file used in this project must have the following structure:

- The **sheet** must be called `Sheet1`.  
- The **columns** should start from the **2nd row** with these headers:

|   | A              | B       | C   | D       | E          | F         |
|---|----------------|---------|-----|---------|------------|-----------|
| 1 |                |         |     |         |            |           |
| 2 | **Nombre Comercial** | **Calle** | **No.** | **Sector** | **Municipio** | **Provincia** |
| 3 | Example Name    | Example St. | 123 | Sector 1 | City 1     | Province 1 |
| 4 | Example Name 2  | Another St. | 456 | Sector 2 | City 2     | Province 2 |
| 5 | ...             | ...     | ... | ...     | ...        | ...       |

- Data should start from **row 3**. The **columns** are as follows:
  - **Nombre Comercial** (Business Name)  
  - **Calle** (Street)  
  - **No.** (Street Number)  
  - **Sector** (Sector)  
  - **Municipio** (City)  
  - **Provincia** (Province)  

---

💡 **This project streamlines visit planning and route optimization, ensuring efficiency in travel.** 🚀

## Installs

In [1]:
!pip install pandas googlemaps ortools folium openpyxl


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 16.3 MB/s eta 0:00:00
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40715 sha256=465f47336dfa22c4858308eb79d77256188bb047f6381d65dc06ab109bf2ca02
  Stored in directory: /root/.cache/pip/wheels/f1/09/77/3cc2f5659cbc62341b30f806aca2b25e6a26c351daa5b1f49a
Successfully built googlemaps
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0


In [2]:

!pip install folium geopy numpy scikit-learn

## Carga de datos

In [ ]:
import pandas as pd
from google.colab import files

# Subir archivo
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
df = pd.read_excel(file_name, sheet_name="Sheet1",header=1)

In [ ]:
df.head()

## Limpieza de datos

In [ ]:
print(df.columns.tolist())

In [ ]:
df.info()

In [ ]:


# Crear columna 'Ubicacion' respetando NaNs originales
columnas_a_combinar = [
    "Calle ",
    "No.",
    "Sector ",
    "Municipio ",
    "Provincia "
]

df["Ubicacion"] = df[columnas_a_combinar].apply(
    lambda row: ', '.join([str(val) for val in row if pd.notna(val)]),
    axis=1
)

# Verificar resultado
df[["Ubicacion"] + columnas_a_combinar].head(10)

## Conseguir Coordenadas

In [ ]:
import googlemaps
import pandas as pd
import time
import logging
import json
from googlemaps.exceptions import ApiError, HTTPError, Timeout, TransportError

# Configuración inicial
API_KEY = "Introduce your API Keys"
gmaps = googlemaps.Client(key=API_KEY)

# Configurar logging
logging.basicConfig(
    filename='geocoding.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Parámetros de ejecución
BATCH_SIZE = 50  # Tamaño de lote para procesamiento
DELAY = 2  # Segundos entre lotes
MAX_RETRIES = 3  # Reintentos por fallo

# Cargar cache si existe
try:
    with open('geocoding_cache.json', 'r') as f:
        cache = json.load(f)
except FileNotFoundError:
    cache = {}

def limpiar_nombres_columnas(df):
    """Elimina espacios adicionales en los nombres de las columnas"""
    df.columns = df.columns.str.strip()
    return df

def limpiar_datos(df):
    """Preprocesamiento de datos críticos"""
    # Llenar valores esenciales y limpiar espacios
    df['Provincia'] = df['Provincia'].fillna('').str.strip().str[:20]
    df['Municipio'] = df['Municipio'].fillna('').str.strip().str[:20]
    df['Calle'] = df['Calle'].fillna('').str.strip().str[:50]
    return df

def construir_direccion(row):
    """Construcción optimizada de dirección con componentes"""
    componentes = {
        'calle': f"{row['Calle']} {row['No.']}".strip() if pd.notna(row['No.']) else row['Calle'],
        'sector': row['Sector'] if pd.notna(row['Sector']) else '',
        'municipio': row['Municipio'],
        'provincia': row['Provincia']
    }

    # Filtrar componentes vacíos
    return {k: v for k, v in componentes.items() if v}

def geocodificar_con_reintentos(componentes, intento=1):
    """Lógica de geocodificación con reintentos inteligentes"""
    try:
        resultado = gmaps.geocode(
            language='es',
            components={
                'route': componentes.get('calle', ''),
                'sublocality': componentes.get('sector', ''),
                'locality': componentes.get('municipio', ''),
                'administrative_area': componentes.get('provincia', ''),
                'country': 'DO'
            }
        )

        if resultado:
            loc = resultado[0]['geometry']['location']
            if 17.4 < loc['lat'] < 19.9 and -72.0 < loc['lng'] < -68.3:
                return loc['lat'], loc['lng']

        # Fallback a municipio-provincia
        if intento == 1 and componentes.get('municipio') and componentes.get('provincia'):
            time.sleep(1)
            return geocodificar_con_reintentos({
                'municipio': componentes['municipio'],
                'provincia': componentes['provincia']
            }, intento=2)

        return None, None

    except (ApiError, HTTPError, Timeout, TransportError) as e:
        if intento < MAX_RETRIES:
            sleep_time = 2 ** intento
            logging.warning(f"Reintento {intento} en {sleep_time}s: {e}")
            time.sleep(sleep_time)
            return geocodificar_con_reintentos(componentes, intento + 1)
        logging.error(f"Fallo definitivo: {e}")
        return None, None

def geocodificar_con_cache(componentes):
    """Geocodificación con cache para evitar solicitudes repetidas"""
    clave_cache = json.dumps(componentes, sort_keys=True)
    if clave_cache in cache:
        return cache[clave_cache]

    lat, lng = geocodificar_con_reintentos(componentes)
    cache[clave_cache] = (lat, lng)

    # Guardar cache actualizado
    with open('geocoding_cache.json', 'w') as f:
        json.dump(cache, f)

    return lat, lng

# Preprocesamiento
df = limpiar_nombres_columnas(df)  # Corregir nombres de columnas
df = limpiar_datos(df)  # Limpiar datos críticos

# Procesar solo direcciones únicas
direcciones_unicas = df[['Calle', 'No.', 'Sector', 'Municipio', 'Provincia']].drop_duplicates()
coordenadas_unicas = {}

for idx, row in direcciones_unicas.iterrows():
    componentes = construir_direccion(row)
    if not componentes.get('provincia') or not componentes.get('municipio'):
        logging.warning(f"Fila {idx}: Datos insuficientes")
        continue

    coordenadas_unicas[json.dumps(componentes, sort_keys=True)] = geocodificar_con_cache(componentes)

# Asignar coordenadas al DataFrame original
def obtener_coordenadas(row):
    componentes = construir_direccion(row)
    clave = json.dumps(componentes, sort_keys=True)
    return coordenadas_unicas.get(clave, (None, None))

df[['Latitud', 'Longitud']] = df.apply(obtener_coordenadas, axis=1, result_type='expand')

# Guardar resultados
df.to_excel("direcciones_con_coordenadas_final.xlsx", index=False)

print("Proceso completado")
print(f"Éxitos: {df[['Latitud', 'Longitud']].notnull().all(axis=1).sum()}")
print(f"Fallos: {df[['Latitud', 'Longitud']].isnull().any(axis=1).sum()}")

In [ ]:
fallos = df[df['Latitud'].isnull() | df['Longitud'].isnull()]
print(fallos[['Calle', 'No.', 'Sector', 'Municipio', 'Provincia']])

In [ ]:
df.info()

## Predecir Rutas

In [ ]:


import numpy as np
from geopy.distance import geodesic
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
import folium
from folium import plugins
from IPython.display import display

# --- FILTRADO Y PREPARACIÓN DE DATOS ---
valid_coords_mask = df[['Latitud', 'Longitud']].notnull().all(axis=1)
valid_coords_df = df[valid_coords_mask].copy()
coordenadas = np.array(list(zip(valid_coords_df["Latitud"], valid_coords_df["Longitud"])))

# --- CLUSTERING CON K-MEANS ---
num_clusters = 10  # Se desean 10 clústeres en total
kmeans = KMeans(n_clusters=num_clusters, n_init=10, random_state=42)
labels = kmeans.fit_predict(coordenadas)
centroids = kmeans.cluster_centers_

# ====================== NUEVO: REBALANCEAR CLÚSTERES PARA QUE NINGUNO EXCEDA 281 PUNTOS ======================
capacidad = 281  # Máximo de puntos por clúster

# Crear diccionario: clave = número de clúster, valor = lista de índices de puntos
clusters_dict = {i: [] for i in range(num_clusters)}
for idx, label in enumerate(labels):
    clusters_dict[label].append(idx)

# Iterar mientras se puedan hacer cambios
cambios = True
while cambios:
    cambios = False
    for cl in range(num_clusters):
        if len(clusters_dict[cl]) > capacidad:
            # Ordenar los puntos del clúster de mayor a menor distancia respecto al centroide
            points_sorted = sorted(clusters_dict[cl],
                                   key=lambda i: np.linalg.norm(coordenadas[i] - centroids[cl]),
                                   reverse=True)
            for point_idx in points_sorted:
                if len(clusters_dict[cl]) <= capacidad:
                    break
                # Calcular las distancias del punto a todos los centroides
                dists_to_all = np.linalg.norm(centroids - coordenadas[point_idx], axis=1)
                candidatos = np.argsort(dists_to_all)
                # Buscar el primer clúster candidato que no sea el actual y que tenga capacidad
                for cand in candidatos:
                    if cand != cl and len(clusters_dict[cand]) < capacidad:
                        clusters_dict[cand].append(point_idx)
                        clusters_dict[cl].remove(point_idx)
                        cambios = True
                        break

# Reconstruir el vector de etiquetas a partir del diccionario rebalaceado
new_labels = np.empty_like(labels)
for cl in range(num_clusters):
    for idx in clusters_dict[cl]:
        new_labels[idx] = cl
labels = new_labels
# ===================================================================================================================

# --- ALGORITMO GENÉTICO OPTIMIZADO PARA TSP ---
def tsp_genetico_optimizado(coordenadas, poblacion_size=30, generaciones=100, tasa_mutacion=0.1):
    n = len(coordenadas)

    # Si hay menos de 3 puntos, retornar la ruta trivial
    if n < 3:
        return list(range(n))

    # Precalcular matriz de distancias (Euclideana aproximada en km)
    dist_matrix = pairwise_distances(coordenadas) * 111  # 1° ≈ 111 km
    np.fill_diagonal(dist_matrix, 0)

    def calcular_fitness(ruta):
        distancia_total = np.sum(dist_matrix[ruta[:-1], ruta[1:]])
        return 1 / distancia_total if distancia_total > 0 else 1e-6  # evitar división por cero

    def generar_individuo():
        return np.random.permutation(n).tolist()

    poblacion = [generar_individuo() for _ in range(poblacion_size)]

    for _ in range(generaciones):
        # Selección por torneo
        padres = []
        for _ in range(poblacion_size):
            indices = np.random.choice(len(poblacion), 3, replace=False)
            candidatos = [poblacion[i] for i in indices]
            padres.append(max(candidatos, key=lambda x: calcular_fitness(x)))

        # Cruzamiento OX
        nueva_poblacion = []
        for i in range(0, poblacion_size, 2):
            p1, p2 = padres[i], padres[i+1]
            punto_cruce = np.random.randint(1, n-1)
            hijo = p1[:punto_cruce] + [g for g in p2 if g not in p1[:punto_cruce]]
            nueva_poblacion.append(hijo)
            nueva_poblacion.append(p2)

        # Mutación
        for ind in nueva_poblacion:
            if np.random.rand() < tasa_mutacion:
                i, j = np.random.choice(n, 2, replace=False)
                ind[i], ind[j] = ind[j], ind[i]

    return max(poblacion, key=lambda x: calcular_fitness(x))

# --- OPTIMIZAR RUTAS POR CLÚSTER ---
rutas_optimas = []
for cluster_id in range(num_clusters):
    # Filtrar coordenadas del clúster según las etiquetas rebalaceadas
    mascara_cluster = (labels == cluster_id)
    coords_cluster = coordenadas[mascara_cluster]

    # Optimizar ruta solo si hay más de 1 punto
    if len(coords_cluster) > 1:
        ruta_optimizada = tsp_genetico_optimizado(coords_cluster)
        rutas_optimas.append(ruta_optimizada)
    else:
        rutas_optimas.append([])

# --- VISUALIZACIÓN ---
# Configurar mapa centrado en República Dominicana
mapa = folium.Map(location=[18.47, -69.93], zoom_start=9)
plugins.Fullscreen().add_to(mapa)

# Paleta de colores para clusters
colores = [
    '#FF0000', '#0000FF', '#00FF00', '#FFA500', '#800080',
    '#FFC0CB', '#008080', '#A52A2A', '#808000', '#00FFFF'
]
# Si se necesitan más colores, se añaden
if num_clusters > len(colores):
    import random
    while len(colores) < num_clusters:
        colores.append("#" + "".join([random.choice('0123456789ABCDEF') for _ in range(6)]))

# Añadir rutas al mapa
for cluster_id, ruta in enumerate(rutas_optimas):
    if len(ruta) < 2:
        continue

    # Obtener puntos del clúster
    mascara_cluster = (labels == cluster_id)
    coords_cluster = coordenadas[mascara_cluster]

    # Crear línea de ruta
    puntos_ruta = [coords_cluster[i] for i in ruta]
    folium.PolyLine(
        locations=puntos_ruta + [puntos_ruta[0]],
        color=colores[cluster_id],
        weight=2,
        opacity=0.7,
        tooltip=f'Agente {cluster_id+1} - {len(ruta)} puntos'
    ).add_to(mapa)

# Ajustar vista a todos los puntos
mapa.fit_bounds([[coordenadas[:, 0].min(), coordenadas[:, 1].min()],
                 [coordenadas[:, 0].max(), coordenadas[:, 1].max()]])

# Mostrar mapa
mapa.get_root().width = "1000px"
mapa.get_root().height = "700px"
display(mapa)

# --- GENERACIÓN DE TABLAS ---
for cluster_id, ruta in enumerate(rutas_optimas):
    if len(ruta) == 0:
        continue

    # Obtener índices originales
    mascara_cluster = (labels == cluster_id)
    df_cluster = valid_coords_df[mascara_cluster].iloc[ruta]

    print(f"\n=== Ruta Agente {cluster_id+1} ({len(ruta)} puntos) ===")
    display(df_cluster[['Nombre Comercial', 'Latitud', 'Longitud']].head())


## Deploy

In [ ]:
import folium
import pandas as pd
import numpy as np
from geopy.distance import geodesic

# --------------------- FUNCIÓN: ORDENAR POR DISTANCIA MÁS CERCANA ---------------------
def ordenar_por_distancia(coords):
    """
    Implementa el algoritmo de Vecino Más Cercano para encontrar la ruta óptima.
    """
    if len(coords) == 0:
        return []

    n = len(coords)
    visitado = np.zeros(n, dtype=bool)
    ruta = [0]  # Empezamos desde el primer punto
    visitado[0] = True

    for _ in range(n - 1):
        ult_punto = coords[ruta[-1]]
        distancias = [
            geodesic(ult_punto, coords[i]).kilometers if not visitado[i] else np.inf
            for i in range(n)
        ]
        siguiente = np.argmin(distancias)
        ruta.append(siguiente)
        visitado[siguiente] = True

    return ruta

# --------------------- BLOQUE: EXTRAER RUTA PARA UN AGENTE Y MOSTRAR MAPA ---------------------
# Solicitar el número del agente (entre 1 y 10)
agent_number = int(input("Ingrese el número del agente (1 a 10): "))
cluster_id = agent_number - 1

# Filtrar los puntos correspondientes al clúster del agente
mask = (labels == cluster_id)
if mask.sum() == 0:
    print(f"No hay puntos asignados para el agente {agent_number}.")
else:
    # Extraer la tabla del clúster
    df_cluster = valid_coords_df[mask].copy()

    # Extraer las coordenadas
    coords_cluster = coordenadas[mask]

    # Ordenar la ruta por distancia más cercana
    ruta_ordenada = ordenar_por_distancia(coords_cluster)

    # Aplicar el orden a la tabla
    df_ruta = df_cluster.iloc[ruta_ordenada].copy()
    df_ruta.reset_index(drop=True, inplace=True)

    # Agregar una columna "Orden" para indicar el número de visita
    df_ruta["Orden"] = df_ruta.index + 1

    # Mostrar la tabla ordenada
    display(df_ruta[['Orden', 'Nombre Comercial', 'Latitud', 'Longitud']])

    # Exportar la tabla a CSV
    csv_filename = f"ruta_agente_{agent_number}.csv"
    df_ruta.to_csv(csv_filename, index=False)
    print(f"CSV generado: {csv_filename}")

    # Crear un mapa centrado en el primer punto de la ruta
    m = folium.Map(location=[coords_cluster[0][0], coords_cluster[0][1]], zoom_start=12)

    # Color único para los puntos del agente
    color_ruta = "blue"  # Puedes cambiar el color aquí

    # Agregar marcadores numerados con el mismo color
    for i, (lat, lon) in enumerate(df_ruta[['Latitud', 'Longitud']].values):
        folium.Marker(
            location=[lat, lon],
            icon=folium.Icon(color=color_ruta, icon="info-sign"),
            popup=f"Orden {i+1}: {df_ruta.iloc[i]['Nombre Comercial']}"
        ).add_to(m)

    # Mostrar el mapa
    display(m)
